In [1]:
from __future__ import print_function
from bs4 import BeautifulSoup
import requests
from six.moves.urllib import parse

In [29]:
base = "https://mapi.yummly.com/mapi/v17/content/feed?tag=list.recipe.other.guided&start={}&limit=10"

r = requests.get(base.format(0))

In [30]:
a = r.json()

In [31]:
len(a['feed'])

10

In [33]:
a['feed'][0].keys()

dict_keys(['display', 'type', 'promoted', 'proRecipe', 'tracking-id', 'seo', 'content'])

In [43]:
import json

In [45]:
with open ('/hri/localdisk/nnabizad/toolpreddata/yummly.json', 'r') as f:
    file = json.load(f)

In [87]:
file[0]['display']['displayName']

'Broccoli and Brown Rice Bowls with Poached Eggs and Red Chilies'

In [53]:
file[0]['content'].keys()

dict_keys(['description', 'urbSubmitters', 'tags', 'preparationSteps', 'videos', 'moreContent', 'tagsAds', 'details', 'relatedContent', 'ingredientLines', 'unitSystem', 'reviews', 'relatedProducts', 'preparationStepCount', 'nutrition', 'yums', 'guidedVariations'])

In [73]:
file[0]['content']

{'description': {'mobileSectionName': 'Description',
  'text': '"Putting an egg on it" makes most dishes more delicious, and this healthy one-bowl meal is no exception. With saltiness from soy sauce, sweetness from brown sugar, crunch from tender-crisp broccoli, and a hint of fire from chiles, the dish will satisfy every tastebud. The recipe is a Yummly original created by Ashley Strickland Freeman.',
  'shortText': None},
 'urbSubmitters': [],
 'tags': {'cuisine': [{'display-name': 'Asian',
    'tag-url': 'cuisine^cuisine-asian'}],
  'nutrition': [{'display-name': 'Source of Omega-3s',
    'tag-url': 'nutrition^nutrition-source-omega-3'},
   {'display-name': 'Low Saturated Fat',
    'tag-url': 'nutrition^nutrition-low-saturated-fat'},
   {'display-name': 'High Vitamin C',
    'tag-url': 'nutrition^nutrition-high-vitamin-c'},
   {'display-name': 'High Protein',
    'tag-url': 'nutrition^nutrition-high-protein'},
   {'display-name': 'High Potassium',
    'tag-url': 'nutrition^nutrition-

In [65]:
file[0]['content']['guidedVariations'][0].keys()

dict_keys(['id', 'default', 'variationAttributes', 'connected', 'actions'])

In [85]:
file[0]['content']['guidedVariations'][0]['actions'][1]['stepGroups'][0]['steps'][0]['equipment']

[{'id': '98674587-cad7-4bcc-926d-41845612c5ea', 'name': 'chef’s knife'},
 {'id': '85acedac-8537-499b-880e-12a751c2b483', 'name': 'cutting board'}]

In [110]:
import pickle as pk
def save_obj(obj, name):
    """
    Saving the pickle object
    """
    with open(name + '.pkl', 'wb') as file:
        pk.dump(obj, file, pk.HIGHEST_PROTOCOL)

In [88]:
titles = []
recs = []
for rec in file:
    titles.append(rec['display']['displayName'])

In [128]:
def clean(txt):
    if 'x13' in txt:txt = '9x13-inch baking dish'
    return(txt)

In [142]:
recs = []
titles = []
for rec in file:
    _tmp = []
    if 'guidedVariations' in rec['content'].keys():
        for step in rec['content']['guidedVariations'][0]['actions']:
            if 'equipment' in step['stepGroups'][0]['steps'][0]:
                eqs = tuple([clean(i['name']) for i in step['stepGroups'][0]['steps'][0]['equipment']])
                _tmp.append(eqs)
        if len(_tmp)>0:
            recs.append(_tmp)
            titles.append(rec['display']['displayName'])

In [143]:
len(recs)

659

In [144]:
len(titles)

659

In [145]:
save_obj(recs, 'recepies')

In [146]:
save_obj(titles, 'titles')

In [147]:
recs

[[('measuring cups', 'medium saucepan'),
  ('chef’s knife', 'cutting board'),
  ('cutting board', 'chef’s knife', 'measuring spoons', 'frying pan', 'lid'),
  ('cutting board', 'chef’s knife', 'serving bowls', 'spoon')],
 [('oven',),
  ('9x13-inch baking dish',),
  ('skillet', 'stove', 'rubber spatula'),
  ('large plate', 'paper towels', 'skillet'),
  ('oven', 'oven mitts', 'toothpick'),
  ('oven', 'oven mitts', 'toothpick'),
  ('oven mitts', 'knife', 'spatula', 'serving spoon')],
 [('kitchen scale', 'measuring cups', 'microwave-safe bowls'),
  ('oven',),
  ('cookie dough scoop',
   'baking sheets',
   'silicone baking mat',
   'parchment paper'),
  ('oven', 'oven mitts'),
  ('oven', 'oven mitts'),
  ('oven mitts', 'spatula', 'cooling rack')],
 [('9x5-inch loaf pan',),
  ('oven mitts', 'toothpick'),
  ('oven mitts', 'toothpicks'),
  ('oven mitts', 'cooling rack'),
  ('knife',)],
 [('knife', 'cutting board'),
  ('oven mitts', 'tooth pick'),
  ('oven mitts', 'toothpick'),
  ('oven mitts',

In [157]:
from collections import Counter

biglis = recs
biglisencoded = []
min_tool_freq = 1
flatlist = []
for lis in biglis:
    [flatlist.append(i) for i in lis]
counts = sorted(Counter(flatlist).items(), key=lambda x: x[1], reverse=True)
encod = dict()
for i, j in enumerate(counts):
    if j[1] > min_tool_freq:
        encod[j[0]] = i + 2
morethanmin = len(encod)
encod['start'] = 1
encod['unknown'] = morethanmin + 1
encod['end'] = morethanmin + 2
decode = {v: k for k, v in encod.items()}
for lis in biglis:
    _tmp = []
    _tmp.append(encod['start'])  # beg
    for i in lis:
        if i in encod:
            _tmp.append(encod[i])
        else:
            _tmp.append(encod['unknown'])  # unk
    _tmp.append(encod['end'])  ##end
    biglisencoded.append(_tmp)

In [159]:
counts

[(('oven mitts',), 616),
 (('oven',), 179),
 (('oven', 'oven mitts'), 146),
 (('oven mitts', 'toothpick'), 129),
 (('baking sheet', 'parchment paper'), 117),
 (('oven mitts', 'meat thermometer'), 77),
 (('oven mitts', 'fork'), 74),
 (('9x13-inch baking dish',), 71),
 (('oven', 'oven mitts', 'toothpick'), 58),
 (('oven', 'oven mitts', 'meat thermometer'), 58),
 (('knife', 'cutting board'), 42),
 (('serving spoon',), 33),
 (('spatula',), 33),
 (('sheet pan', 'parchment paper'), 28),
 (('baking sheet', 'aluminum foil'), 22),
 (('toothpick', 'oven mitts'), 20),
 (('plate',), 19),
 (('oven mitts', 'serving spoon'), 18),
 (('knife',), 16),
 (('oven', 'oven mitts', 'fork'), 16),
 (('9-inch square pan',), 15),
 (('baking sheets', 'parchment paper', 'silicone baking mat'), 15),
 (('oven mitts', 'cooling rack', 'spatula'), 15),
 (('roasting pan', 'roasting rack', 'aluminum foil'), 15),
 (('oven mitts', 'knife'), 14),
 (('muffin liners', 'muffin pan'), 14),
 (('large pot', 'colander'), 14),
 (('s

In [161]:
len(biglisencoded)

659

In [162]:
decode

{2: ('oven mitts',),
 3: ('oven',),
 4: ('oven', 'oven mitts'),
 5: ('oven mitts', 'toothpick'),
 6: ('baking sheet', 'parchment paper'),
 7: ('oven mitts', 'meat thermometer'),
 8: ('oven mitts', 'fork'),
 9: ('9x13-inch baking dish',),
 10: ('oven', 'oven mitts', 'toothpick'),
 11: ('oven', 'oven mitts', 'meat thermometer'),
 12: ('knife', 'cutting board'),
 13: ('serving spoon',),
 14: ('spatula',),
 15: ('sheet pan', 'parchment paper'),
 16: ('baking sheet', 'aluminum foil'),
 17: ('toothpick', 'oven mitts'),
 18: ('plate',),
 19: ('oven mitts', 'serving spoon'),
 20: ('knife',),
 21: ('oven', 'oven mitts', 'fork'),
 22: ('9-inch square pan',),
 23: ('baking sheets', 'parchment paper', 'silicone baking mat'),
 24: ('oven mitts', 'cooling rack', 'spatula'),
 25: ('roasting pan', 'roasting rack', 'aluminum foil'),
 26: ('oven mitts', 'knife'),
 27: ('muffin liners', 'muffin pan'),
 28: ('large pot', 'colander'),
 29: ('square baking dish',),
 30: ('chef’s knife', 'cutting board'),
 3

In [163]:
encod

{('oven mitts',): 2,
 ('oven',): 3,
 ('oven', 'oven mitts'): 4,
 ('oven mitts', 'toothpick'): 5,
 ('baking sheet', 'parchment paper'): 6,
 ('oven mitts', 'meat thermometer'): 7,
 ('oven mitts', 'fork'): 8,
 ('9x13-inch baking dish',): 9,
 ('oven', 'oven mitts', 'toothpick'): 10,
 ('oven', 'oven mitts', 'meat thermometer'): 11,
 ('knife', 'cutting board'): 12,
 ('serving spoon',): 13,
 ('spatula',): 14,
 ('sheet pan', 'parchment paper'): 15,
 ('baking sheet', 'aluminum foil'): 16,
 ('toothpick', 'oven mitts'): 17,
 ('plate',): 18,
 ('oven mitts', 'serving spoon'): 19,
 ('knife',): 20,
 ('oven', 'oven mitts', 'fork'): 21,
 ('9-inch square pan',): 22,
 ('baking sheets', 'parchment paper', 'silicone baking mat'): 23,
 ('oven mitts', 'cooling rack', 'spatula'): 24,
 ('roasting pan', 'roasting rack', 'aluminum foil'): 25,
 ('oven mitts', 'knife'): 26,
 ('muffin liners', 'muffin pan'): 27,
 ('large pot', 'colander'): 28,
 ('square baking dish',): 29,
 ('chef’s knife', 'cutting board'): 30,
 (

In [164]:
save_obj(biglisencoded, 'yammly_encoded_tools')
save_obj(encod, 'yammly_encodedic')
save_obj(decod, 'yammly_decodedic')

In [165]:
decod

{2: ('oven mitts',),
 3: ('oven',),
 4: ('oven', 'oven mitts'),
 5: ('oven mitts', 'toothpick'),
 6: ('baking sheet', 'parchment paper'),
 7: ('oven mitts', 'meat thermometer'),
 8: ('oven mitts', 'fork'),
 9: ('9x13-inch baking dish',),
 10: ('oven', 'oven mitts', 'toothpick'),
 11: ('oven', 'oven mitts', 'meat thermometer'),
 12: ('knife', 'cutting board'),
 13: ('serving spoon',),
 14: ('spatula',),
 15: ('sheet pan', 'parchment paper'),
 16: ('baking sheet', 'aluminum foil'),
 17: ('toothpick', 'oven mitts'),
 18: ('plate',),
 19: ('oven mitts', 'serving spoon'),
 20: ('knife',),
 21: ('oven', 'oven mitts', 'fork'),
 22: ('9-inch square pan',),
 23: ('baking sheets', 'parchment paper', 'silicone baking mat'),
 24: ('oven mitts', 'cooling rack', 'spatula'),
 25: ('roasting pan', 'roasting rack', 'aluminum foil'),
 26: ('oven mitts', 'knife'),
 27: ('muffin liners', 'muffin pan'),
 28: ('large pot', 'colander'),
 29: ('square baking dish',),
 30: ('chef’s knife', 'cutting board'),
 3

In [167]:
ntitle = []
for i in titles:
    ntitle.append([i])

In [168]:
ntitle

[['Broccoli and Brown Rice Bowls with Poached Eggs and Red Chilies'],
 ['Sausage and Cheese Egg Casserole'],
 ['Brown Sugar Cookies'],
 ['Gluten-Free Zucchini Bread'],
 ['Herb Roasted Garlic'],
 ['Smoky Roasted Fall Vegetables'],
 ['Vegan Black Bean Enchiladas'],
 ['Asian Baked Chicken Legs'],
 ['Keto Magic Coconut Macaroons'],
 ['One-Bowl Pumpkin Sheet Cake with Chocolate Bourbon Buttercream'],
 ['Vegan Bread Pudding'],
 ['Individual Garlic Parmesan Potatoes Au Gratin'],
 ['Easy Ground Beef Enchiladas'],
 ['Honey Garlic Baked Shrimp'],
 ['Baked Chicken Nuggets'],
 ['Korean Bulgogi'],
 ['Bacon, Potato, and Egg Casserole Bites'],
 ['Basic Vanilla Cupcakes with Vanilla Buttercream'],
 ['Grilled Cheese Pinwheels'],
 ['Sugar-Free Red Velvet Cake'],
 ['Christmas Sugar Cookies'],
 ['Cajun Shrimp, Sausage, and Vegetable Sheet Pan'],
 ['Reduced Carb Cauliflower & Zucchini Tots'],
 ['Ham and Cheese Quiche'],
 ['Sheet Pan Black Bean and Vegetable Breakfast Burritos'],
 ['Apple Cinnamon Coffee Ca

In [169]:
save_obj(ntitle, 'cats')